In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adam
import cv2

In [2]:
# Set the root directory of the image database
root_dir = "/EarVN1dataset/images"

In [3]:
def preprocess_image(img):
    # Resize the image to a fixed size
    image = cv2.resize(img, (224, 224))

    # Convert the color space from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert the pixel values to floats between 0 and 1
    image = image.astype("float32") / 255.0

    # Subtract the mean RGB values of the ImageNet dataset
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image -= mean
    image /= std

    # Add a batch dimension to the image
    #image = image.reshape((1, *image.shape))

    return image

In [5]:
# Get the filenames and labels of all the images in the database
filenames = []
X=[]
labels = []
for root,dirs,files in os.walk(root_dir):          
    # Check if the item is a file
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            img=cv2.imread(image_path) 
            img=preprocess_image(img)
            # Get the label of the image from the subdirectory name
            label =file.split(' ')[0]
            X.append(img)
            filenames.append(os.path.join(root_dir, file))
            labels.append(label)
          
      
      
      
      
      


In [ ]:
print(X[7].shape)

(224, 224, 3)


In [6]:
# Convert the label data to integers
labels = np.array(labels)

In [7]:
label_encoder = LabelEncoder()
integer_labels = label_encoder.fit_transform(labels)


In [8]:
# Convert the integer labels to one-hot encoded form
onehot_encoder = OneHotEncoder(sparse=False)
labels = onehot_encoder.fit_transform(integer_labels.reshape(-1, 1))

In [ ]:
print(len(X))

28412


In [ ]:
X_=[]
for elem in X:
    if elem is not None:
        X_.append(elem)

In [ ]:
print(len(X_))

28412


In [9]:
# Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [10]:
# Load the VGG16 model, excluding the top fully connected layers
vgg16 = keras.applications.vgg16.VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

In [11]:
# Add a flatten layer and a fully connected layer on top of the VGG16 model
model = keras.Sequential([
    vgg16,
    layers.Flatten(),
    
    layers.Dense(4096, activation="relu"),
    layers.Dense(164, activation="softmax")
])


In [12]:
# Freeze the weights of the VGG16 layers
for layer in vgg16.layers:
    layer.trainable = False


In [13]:
def image_generator(images, labels, batch_size):
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    while True:
        # Shuffle the data
        indices = np.random.permutation(len(images))

        '''if len(indices.shape) > 1:
            # If the permutation function returns a matrix, select a random index from the matrix
            index = np.random.randint(0, len(indices))
            
        else:
            #If the permutation function returns a single value, use it as the index
            index = np.random.randint(0, len(indices))
        filenames = filenames[index]
        labels = labels[index]'''

        
        
        # Divide the data into batches
        for i in range(0, len(images), batch_size):
            batch_images = images[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]
            
            # Load and preprocess the images in the batch
            

            
            no= datagen.flow(batch_images, batch_labels, batch_size=batch_size,save_to_dir=None, save_prefix='')
            images_batch, labels_batch= next(no)
                
            yield images_batch, labels_batch

            

In [14]:
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)


In [15]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

In [16]:
y_train=np.array(y_train)
y_val=np.array(y_val)

In [17]:
y_train = keras.utils.to_categorical(y_train, num_classes=164)
y_val = keras.utils.to_categorical(y_val, num_classes=164)

In [ ]:
# Train the model
batch_size = 8
num_epochs = 30
X_train = np.array(X_train)
X_val = np.array(X_val)
# Create the image generators for the training and validation sets
train_generator = image_generator(X_train, y_train, batch_size)
val_generator = image_generator(X_val, y_val, batch_size)




In [ ]:
print(y_val.shape)

(4546, 181)


In [ ]:
print(X_train[0].shape)

(43, 32, 3)


In [ ]:
print(labels.shape)


(28412,)


In [ ]:
# Train the model
history = model.fit(train_generator, epochs=num_epochs, steps_per_epoch=len(X_train)//batch_size,
                    validation_data=val_generator, validation_steps=len(X_val)//batch_size)

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from keras.preprocessing import image
from PIL import Image
import cv2
from skimage.feature import hog

In [ ]:


# Set the path to the dataset
dataset_path = '/content/drive/MyDrive/EarDS/'

# Create an empty list to store the labels
labels = []

# Iterate over the files in the dataset directory
for file in os.listdir(dataset_path):
    # Extract the label from the filename
    label = file.split('_')[0]  # Assumes that the label is the first part of the filename, separated by an underscore
    labels.append(label)

# Convert the list of labels to a NumPy array
labels = np.array(labels)

# Save the labels to a file
np.save('ear_labels.npy', labels)

In [ ]:
images = []

# Iterate over the files in the dataset directory
for file in os.listdir(dataset_path):
    # Load the image
    image = Image.open(os.path.join(dataset_path, file))

    # Convert the image to a NumPy array
    image = np.array(image)

    # Add the image to the list
    images.append(image)

# Convert the list of images to a NumPy array
images = np.array(images)

# Save the images to a .npy file
np.save('ear_images.npy', images)


In [ ]:
def extract_features(model, ear_image):
    # Preprocess the image
    processed_image = keras.applications.vgg16.preprocess_input(ear_image)

    # Extract features using the model
    features = model.predict(processed_image)

    # Flatten the features
    features = features.flatten()

    return features

In [ ]:
# Load the ear images and labels
X = np.load('ear_images.npy')
y = np.load('ear_labels.npy')

# Pre-process the data
X = X.astype('float32') / 255.0